# Sentiment Analysis of Element Coffee Shop Reviews

In [1]:
import os.path
import requests
import pickle

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

from transformers import pipeline

### Obtain Data with Google Business API

In [2]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/business.manage']

In [6]:
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
        creds=flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    
    service = build('mybusiness', 'v4', credentials=creds)
    
    # Call the gmb v4 API
    results = f'https://mybusiness.googleapis.com/v4/accounts/{accountId}/locations/{locationId}/reviews?pageSize=50'
    
    if not results:
        print('No files found.')
    else:
        print(results.txt)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=634188717068-unqb3q8t0is3e7nnc2gch1ljd9so0etj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A56743%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbusiness.manage&state=7vcBGDmCOFLxaNrIcDOEAKGuEE1NkD&access_type=offline


HttpError: <HttpError 403 when requesting https://mybusiness.googleapis.com/$discovery/rest?version=v4 returned "The request is missing a valid API key.">

In [31]:


URI = f'https://mybusiness.googleapis.com/v4/accounts/{accountId}/locations/{locationId}/reviews?pageSize=10'
r = requests.get(URI, headers=headers)

In [32]:
r.status_code

200

In [1]:
result = r.json()
result

NameError: name 'r' is not defined

### NLP Sentiment-Analysis with Transformers Library

In [2]:
nlp = pipeline("sentiment-analysis")

In [5]:
result = nlp("Lovely atmosphere, staff are super friendly and wonderful people.")[0]

In [6]:
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9999
